## Install and Import Dependencies

In [ ]:
# Install ultralytics if not already installed
!pip install ultralytics

In [7]:
from ultralytics import YOLO
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.6.0+rocm6.2.4
CUDA available: True
CUDA device: AMD Radeon Graphics


## Load YOLOv11n Model

In [2]:
# Load the YOLOv11n model
model = YOLO('yolo11n.pt')  # Load pre-trained YOLOv11n model
print("Model loaded successfully")

Model loaded successfully


In [2]:
model = YOLO('runs/detect/yolo11n_training5/weights/best.pt')
print("Custom trained model loaded successfully")

Custom trained model loaded successfully


## Configure Training Parameters

In [3]:
# Training configuration
data_yaml = 'data.yaml'
epochs = 50
imgsz = 640
batch_size = 16  # Reduced from 32 to avoid memory issues
project_name = 'runs/detect'
name = 'yolo11n_training'

print(f"Data config: {data_yaml}")
print(f"Epochs: {epochs}")
print(f"Image size: {imgsz}")
print(f"Batch size: {batch_size}")

Data config: data.yaml
Epochs: 50
Image size: 640
Batch size: 16


## Train the Model

In [5]:
# Train the model
results = model.train(
    data=data_yaml,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    project=project_name,
    name=name,
    save=True,
    save_period=10,  # Save checkpoint every 10 epochs
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=8,
    # Augmentations to prevent face misclassification
    mosaic=1.0,        # 4-image mosaic - critical for context learning
    scale=0.5,         # ±50% scale variation
    translate=0.2,     # ±20% translation
    degrees=15,        # MORE rotation - products grabbed at various angles
    flipud=0.0,        # No vertical flips (products/faces won't be upside down)
    fliplr=0.5,        # Horizontal flips OK
    mixup=0.1,         # Light blending
    copy_paste=0.05,   # Minimal copy-paste
    erasing=0.4,       # Random erasing - forces robust features
    hsv_h=0.015,       # Hue variation
    hsv_s=0.7,         # Saturation variation
    hsv_v=0.4          # Brightness variation
)

print("Training completed!")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.6.0+rocm6.2.4 CUDA:0 (AMD Radeon Graphics, 16368MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_training11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

In [4]:
model = YOLO('runs/detect/yolo11n_training11/weights/best.pt')

results = model.train(
    data=data_yaml,
    epochs=10,              # short polishing run
    imgsz=imgsz,
    batch=batch_size,
    project=project_name,
    name=f"{name}_finetune",
    save=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=8,
    lr0=0.5 * 0.01,         # if your original lr0 was 0.01, halve it

    # Softer augmentations for final refinement
    mosaic=0.0,             # disable mosaic in fine-tune
    scale=0.3,              # ±30% instead of 50%
    translate=0.1,          # ±10% instead of 20%
    degrees=10,             # lighter rotation
    flipud=0.0,
    fliplr=0.5,

    mixup=0.0,              # off (no blending)
    copy_paste=0.0,         # off
    erasing=0.15,           # lighter erasing

    hsv_h=0.015,
    hsv_s=0.4,              # reduce saturation jitter
    hsv_v=0.25              # reduce brightness jitter
)


Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.6.0+rocm6.2.4 CUDA:0 (AMD Radeon Graphics, 16368MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.15, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.25, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/yolo11n_training11/weights/best.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolo11n_training_finetune2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tru

/home/aleisley/Documents/nadali/.venv/lib/python3.12/site-packages/ultralytics/nn/modules/block.py:1299: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  attn = (q.transpose(-2, -1) @ k) * self.scale


ERROR ❌ AMP: checks failed. Anomalies were detected with AMP on your system that may lead to NaN losses or zero-mAP results, so AMP will be disabled during training.
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 149.4±74.7 MB/s, size: 48.4 KB)
train: Scanning /home/aleisley/Documents/nadali/train/labels.cache... 16390 images, 5 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 16390/16390 54.6Mit/s 0.0s
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 162.7±58.2 MB/s, size: 35.1 KB)
val: Scanning /home/aleisley/Documents/nadali/test/labels.cache... 907 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 907/907 2.4Mit/s 0.0s0s
Plotting labels to /home/aleisley/Documents/nadali/runs/detect/yolo11n_training_finetune2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000263, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005)

In [9]:
model = YOLO('runs/detect/yolo11n_training_finetune2/weights/best.pt')

results = model.train(
    data=data_yaml,
    epochs=20,                 # short but meaningful fine-tune
    imgsz=imgsz,
    batch=batch_size,
    project=project_name,
    name=f"{name}_fp_reduction",
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=8,

    # LR slightly lower than original (if you set lr0 before)
    lr0=0.5 * 0.01,           # adjust 0.01 to your original lr0

    # AUGMENTATIONS: slightly softer, more realistic
    mosaic=0.7,               # not 1.0; avoid extreme composites
    close_mosaic=10,          # disable mosaic in last 10 epochs (if supported in your version)
    scale=0.3,                # ±30% instead of 50%
    translate=0.1,            # ±10% instead of 20%
    degrees=10,               # small rotations
    flipud=0.0,
    fliplr=0.5,

    mixup=0.0,                # turn off – can create unrealistic blends
    copy_paste=0.0,           # turn off – also can create weird composites
    erasing=0.2,              # keep but softer than 0.4

    hsv_h=0.015,
    hsv_s=0.4,                # reduce from 0.7
    hsv_v=0.25                # reduce from 0.4
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.6.0+rocm6.2.4 CUDA:0 (AMD Radeon Graphics, 16368MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.25, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/yolo11n_training_finetune2/weights/best.pt, momentum=0.937, mosaic=0.7, multi_scale=False, name=yolo11n_training_fp_reduction, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

In [5]:
# For the second wave
metrics = model.val()

print(f"\nValidation Results:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.6.0+rocm6.2.4 CUDA:0 (AMD Radeon Graphics, 16368MiB)
YOLO11n summary (fused): 100 layers, 2,588,782 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2842.5±747.3 MB/s, size: 37.5 KB)
val: Scanning /home/aleisley/Documents/nadali/test/labels.cache... 907 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 907/907 3.9Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 57/57 20.2it/s 2.8s0.0s
                   all        907       2141      0.967      0.957      0.984      0.865
        coffee_nescafe         25         25          1      0.996      0.995      0.897
         coffee_kopiko         25         25      0.981          1      0.995      0.886
Lucky-Me-Pancit-Canton         25         39       0.99          1      0.995      0.948
           Coke-in-can         25         35      0.992          1      0.995       0.98
           

In [5]:
results = model.train(
    resume=True,
    epochs=50
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.6.0+rocm6.2.4 CUDA:0 (AMD Radeon Graphics, 16368MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=runs/detect/yolo11n_training5/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_training5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

/home/aleisley/Documents/nadali/.venv/lib/python3.12/site-packages/ultralytics/nn/modules/block.py:1299: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  attn = (q.transpose(-2, -1) @ k) * self.scale


ERROR ❌ AMP: checks failed. Anomalies were detected with AMP on your system that may lead to NaN losses or zero-mAP results, so AMP will be disabled during training.
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 172.7±84.5 MB/s, size: 33.6 KB)
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 172.7±84.5 MB/s, size: 33.6 KB)
train: Scanning /home/aleisley/Documents/nadali/train/labels.cache... 16876 images, 5 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 16876/16876 61.2Mit/s 0.0s
train: /home/aleisley/Documents/nadali/train/images/coffee0246.jpg: 1 duplicate labels removed
train: /home/aleisley/Documents/nadali/train/images/combined_dewberry_strawberry_349.jpg: 1 duplicate labels removed
train: Scanning /home/aleisley/Documents/nadali/train/labels.cache... 16876 images, 5 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 16876/16876 61.2Mit/s 0.0s
train: /home/aleisley/Documents/nadali/train/images/coffee0246.jpg: 1 duplicate labels removed
train: /home/aleisley/Documents/nadali/train/imag

## Validate the Model

In [6]:
# Validate the model on the test set
metrics = model.val()

print(f"\nValidation Results:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.6.0+rocm6.2.4 CUDA:0 (AMD Radeon Graphics, 16368MiB)
YOLO11n summary (fused): 100 layers, 2,588,782 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4963.2±1867.6 MB/s, size: 36.4 KB)
YOLO11n summary (fused): 100 layers, 2,588,782 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4963.2±1867.6 MB/s, size: 36.4 KB)
val: Scanning /home/aleisley/Documents/nadali/test/labels.cache... 907 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 907/907 3.8Mit/s 0.0s0s
val: Scanning /home/aleisley/Documents/nadali/test/labels.cache... 907 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 907/907 3.8Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 57/57 19.0it/s 3.0s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 57/57 19.0it/s 3.0s
            

## Export Model to PyTorch Format

In [6]:
# Export the trained model to PyTorch format
model_path = model.export(format='torchscript')
print(f"\nModel exported to: {model_path}")

# Also save the best weights
best_model_path = f"{project_name}/{name}/weights/best.pt"
print(f"Best model weights saved at: {best_model_path}")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.6.0+rocm6.2.4 CPU (AMD Ryzen 7 9800X3D 8-Core Processor)

PyTorch: starting from '/home/aleisley/Documents/nadali/runs/detect/yolo11n_training_finetune2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 38, 8400) (5.2 MB)

TorchScript: starting export with torch 2.6.0+rocm6.2.4...

PyTorch: starting from '/home/aleisley/Documents/nadali/runs/detect/yolo11n_training_finetune2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 38, 8400) (5.2 MB)

TorchScript: starting export with torch 2.6.0+rocm6.2.4...
TorchScript: export success ✅ 0.6s, saved as '/home/aleisley/Documents/nadali/runs/detect/yolo11n_training_finetune2/weights/best.torchscript' (10.4 MB)

Export complete (0.6s)
Results saved to /home/aleisley/Documents/nadali/runs/detect/yolo11n_training_finetune2/weights
Predict:         yolo predict task=detect model=/home/aleisley/Documents/nadali/runs/detect/yolo11n_training_finetu

## View Training Results

In [ ]:
# Display training results
from IPython.display import Image, display
import glob

results_dir = f"{project_name}/{name}"
print(f"\nResults saved in: {results_dir}")

# Display result images if available
result_images = glob.glob(f"{results_dir}/*.png")
for img_path in result_images[:5]:  # Show first 5 images
    print(f"\n{os.path.basename(img_path)}:")
    display(Image(filename=img_path, width=600))